In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import association_rules, apriori
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

#https://www.kaggle.com/rmisra/clothing-fit-dataset-for-size-recommendation?select=renttherunway_final_data.json

In [2]:
df_train = pd.read_csv("X_train.csv")
df_test = pd.read_csv("X_test.csv")

In [3]:
df_train.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,13906,34b,2616422,115lbs,10.0,everyday,its loose fitting in a good way,straight & narrow,"loved it, fun blouse, super pretty and easy to...",top,"5' 7""",8,32.0,"January 3, 2018"
1,fit,18490,32c,152836,115lbs,8.0,date,This dress was amazing! It fit like a glove a...,petite,Love it!,mini,"5' 0""",4,37.0,"March 30, 2014"
2,fit,476969,32b,176490,130lbs,10.0,wedding,I loved wearing this dress. It is really struc...,athletic,Love it.,dress,"5' 7""",8,34.0,"April 21, 2016"
3,fit,542220,34c,1777332,138lbs,8.0,work,It's a little boxy and very warm as it is wool...,athletic,"Classy, conservative work dress.",dress,"5' 7""",12,32.0,"February 10, 2016"
4,fit,334094,34a,1547971,130lbs,10.0,wedding,Rented this dress for a black-tie wedding and ...,straight & narrow,"Elegant, interesting, glamorous… can I keep it?",gown,"5' 6""",8,32.0,"March 25, 2016"


## New content (Este es el ultimo)

In [4]:
df_train.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,13906,34b,2616422,115lbs,10.0,everyday,its loose fitting in a good way,straight & narrow,"loved it, fun blouse, super pretty and easy to...",top,"5' 7""",8,32.0,"January 3, 2018"
1,fit,18490,32c,152836,115lbs,8.0,date,This dress was amazing! It fit like a glove a...,petite,Love it!,mini,"5' 0""",4,37.0,"March 30, 2014"
2,fit,476969,32b,176490,130lbs,10.0,wedding,I loved wearing this dress. It is really struc...,athletic,Love it.,dress,"5' 7""",8,34.0,"April 21, 2016"
3,fit,542220,34c,1777332,138lbs,8.0,work,It's a little boxy and very warm as it is wool...,athletic,"Classy, conservative work dress.",dress,"5' 7""",12,32.0,"February 10, 2016"
4,fit,334094,34a,1547971,130lbs,10.0,wedding,Rented this dress for a black-tie wedding and ...,straight & narrow,"Elegant, interesting, glamorous… can I keep it?",gown,"5' 6""",8,32.0,"March 25, 2016"


In [5]:
body_type = []
category = []
rented_for = []

In [6]:
list_unique_id = np.unique(df_train['item_id'])
list_unique_id

array([ 123373,  123793,  124204, ..., 2965009, 2965924, 2966087],
      dtype=int64)

In [7]:
df_for_itemid = df_train[df_train['item_id']== 123373]
df_for_itemid

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
2020,fit,575212,34c,123373,135lbs,8.0,wedding,I LOVED this dress. It fit well and was so bea...,athletic,"Simple and elegant! But, be warned!",gown,"5' 8""",9,33.0,"January 2, 2015"
2358,fit,849571,38c,123373,194lbs,10.0,formal affair,The dress was absolutely amazing! It fit perfe...,hourglass,The dress was the perfect touch to an amazing ...,gown,"5' 7""",29,21.0,"February 3, 2016"
2773,fit,669706,NaN,123373,125lbs,10.0,formal affair,"I ordered both sizes 4 and 6, but opted for th...",petite,Beautiful classy dress for a black tie wedding,gown,"5' 3""",8,40.0,"June 1, 2014"
2775,fit,277450,34a,123373,128lbs,10.0,formal affair,This fits well. The 4 worked perfectly; I thi...,straight & narrow,Hot.,gown,"5' 7""",8,38.0,"December 12, 2014"
3594,fit,342457,NaN,123373,140lbs,10.0,party,Elegant for any type of occasion.,full bust,True size 4 fit amazing wished it was just a l...,gown,"5' 10""",9,33.0,"October 6, 2014"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75879,fit,791053,38b,123373,140lbs,10.0,formal affair,This was for a formal event and it was so clas...,full bust,"I loved wearing this dress, I would definitely...",gown,"5' 2""",15,57.0,"March 7, 2015"
77113,fit,502133,34c,123373,125lbs,10.0,other,"I am in love with this dress seriously, it's b...",straight & narrow,I love this dress so much... I think I might b...,gown,"5' 7""",8,30.0,"December 12, 2017"
77513,small,773974,34d,123373,130lbs,6.0,wedding,Dress was perfect for hiding my tiny bump. I ...,athletic,I am pregnant and this dress hid my growing be...,gown,"5' 6""",16,33.0,"November 18, 2013"
78059,fit,255578,34b,123373,165lbs,8.0,formal affair,i really like the dress it was nice ... but it...,pear,great basic black dress,gown,"5' 5""",20,48.0,"November 23, 2015"


In [8]:
new_df = pd.DataFrame(df_train['rented for'])
new_df['category'] = df_train['category']
new_df['body type']= df_train['body type']
new_df['item_id']= df_train['item_id']
new_df

,rented for,category,body type,item_id
0,everyday,top,straight & narrow,2616422
1,date,mini,petite,152836
2,wedding,dress,athletic,176490
3,work,dress,athletic,1777332
4,wedding,gown,straight & narrow,1547971
...,...,...,...,...
79150,everyday,top,hourglass,2210425
79151,wedding,dress,pear,762257
79152,formal affair,dress,hourglass,221704
79153,party,dress,hourglass,1814016


In [9]:
new_df['soup']= (new_df.values.tolist())
new_df

,rented for,category,body type,item_id,soup
0,everyday,top,straight & narrow,2616422,"[everyday, top, straight & narrow, 2616422]"
1,date,mini,petite,152836,"[date, mini, petite, 152836]"
2,wedding,dress,athletic,176490,"[wedding, dress, athletic, 176490]"
3,work,dress,athletic,1777332,"[work, dress, athletic, 1777332]"
4,wedding,gown,straight & narrow,1547971,"[wedding, gown, straight & narrow, 1547971]"
...,...,...,...,...,...
79150,everyday,top,hourglass,2210425,"[everyday, top, hourglass, 2210425]"
79151,wedding,dress,pear,762257,"[wedding, dress, pear, 762257]"
79152,formal affair,dress,hourglass,221704,"[formal affair, dress, hourglass, 221704]"
79153,party,dress,hourglass,1814016,"[party, dress, hourglass, 1814016]"


In [10]:
new_df['soup']=new_df['soup'].apply(lambda x:" "+" ".join(x[0:3]))
new_df

,rented for,category,body type,item_id,soup
0,everyday,top,straight & narrow,2616422,everyday top straight & narrow
1,date,mini,petite,152836,date mini petite
2,wedding,dress,athletic,176490,wedding dress athletic
3,work,dress,athletic,1777332,work dress athletic
4,wedding,gown,straight & narrow,1547971,wedding gown straight & narrow
...,...,...,...,...,...
79150,everyday,top,hourglass,2210425,everyday top hourglass
79151,wedding,dress,pear,762257,wedding dress pear
79152,formal affair,dress,hourglass,221704,formal affair dress hourglass
79153,party,dress,hourglass,1814016,party dress hourglass


In [11]:
new_df_soup = pd.DataFrame(new_df.groupby('item_id')['soup'].sum())
new_df_soup

,soup
item_id,
123373,wedding gown athletic formal affair gown hour...
123793,party gown hourglass formal affair gown hourg...
124204,wedding dress athletic formal affair dress pe...
124553,wedding dress athletic wedding dress pear wed...
125424,party dress hourglass party dress full bust o...
...,...
2963850,everyday skirt pear everyday skirt full bust ...
2964470,everyday top hourglass
2965009,date coat straight & narrow everyday coat hou...


In [12]:
new_df_soup = new_df_soup.reset_index()
new_df_soup

,item_id,soup
0,123373,wedding gown athletic formal affair gown hour...
1,123793,party gown hourglass formal affair gown hourg...
2,124204,wedding dress athletic formal affair dress pe...
3,124553,wedding dress athletic wedding dress pear wed...
4,125424,party dress hourglass party dress full bust o...
...,...,...
5582,2963850,everyday skirt pear everyday skirt full bust ...
5583,2964470,everyday top hourglass
5584,2965009,date coat straight & narrow everyday coat hou...
5585,2965924,date jacket petite work jacket hourglass


In [13]:
#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer()#stop_words='english'
count_matrix = count.fit_transform(new_df_soup['soup'])

In [14]:
#Compute the cosine similarity score (equivalent to dot product for count vectorizw)
cosine_sim2 = cosine_similarity(count_matrix) #matrix con las transacciones en filas y columnas
len(cosine_sim2)

5587

In [15]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(new_df_soup.index, index=new_df_soup['item_id'])

In [16]:
idx = indices[123373]
idx

0

In [17]:
sim_scores = list(enumerate(cosine_sim2[idx]))
sim_scores

[(0, 1.0),
 (1, 0.9915057941685594),
 (2, 0.28343260165658124),
 (3, 0.2787867331248628),
 (4, 0.24863220817699014),
 (5, 0.9888220294503832),
 (6, 0.9956908464088303),
 (7, 0.34698146185535444),
 (8, 0.9788239411155487),
 (9, 0.2236860984923135),
 (10, 0.9916859159933702),
 (11, 0.30098386373258107),
 (12, 0.9943902634467693),
 (13, 0.268366865406645),
 (14, 0.3531911100073867),
 (15, 0.3750548925978059),
 (16, 0.9369739684382368),
 (17, 0.9537603266763387),
 (18, 0.9898330966180771),
 (19, 0.9894555725103641),
 (20, 0.997433405763327),
 (21, 0.3073846201574533),
 (22, 0.9964858490740153),
 (23, 0.2834023653373213),
 (24, 0.9942376602093523),
 (25, 0.3837308906025051),
 (26, 0.3631094983705756),
 (27, 0.2844646861254103),
 (28, 0.2854235347535041),
 (29, 0.3083645841627313),
 (30, 0.9569350619606762),
 (31, 0.9891728973508656),
 (32, 0.9919205511837275),
 (33, 0.9975954886026817),
 (34, 0.9571209148490893),
 (35, 0.46841100136817415),
 (36, 0.9957115195957376),
 (37, 0.991134875440714

# Recommendation function

In [18]:
#Me quedo con los usuarios que dieron raiting alto para comparar contra mis predicciones
X_test = df_test[df_test["rating"] >= 8]

In [19]:
#Nos quedamos con los usuarios unicos de test para predecir raitings con items
users_test_unique = X_test["user_id"].unique()
len(users_test_unique)

23721

In [20]:
df_train_best_ratings = df_train.sort_values(by= "rating",ascending= False)

In [21]:
def content_recommender_generic(item_id, distance_matrix, n , df=new_df_soup, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[item_id]
    
    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    
    sim_scores = list(enumerate(distance_matrix[idx]))
    
    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:n]
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 most similar movies
    return new_df_soup['item_id'].iloc[movie_indices]

In [22]:
def n_content_recommendation(users_test_unique, n , distance_matrix):
    n = n
    reco=[]
    distance_matrix = distance_matrix
    count_user = 0
    #count_user_colab = 0
    for user_id in users_test_unique: 
        #Me quedo con el primer item relevante del usuario en train
        item_id_content = df_train_best_ratings[df_train_best_ratings["user_id"] == user_id]["item_id"].values[0]
        #Aplico la funcion de content
        content_recommendation = pd.DataFrame(content_recommender_generic(item_id=item_id_content, distance_matrix=distance_matrix, n=n))
        content_recommendation["user_id"] = user_id
        reco.append(content_recommendation["item_id"].to_list())
         
        count_user += 1     
        print("Progress in % {:.2f} " .format((count_user /len(users_test_unique) * 100)))
    
    return reco

# Score metrics (precision, recall and F1)

In [23]:
reco_all_user = []
def score_metrics( n=6, reco_all_user=reco_all_user):
    TP = 0
    FN = 0
    for i in range(len(reco_all_user)):
        user = users_test_unique[i]
        list_test_item_per_user = X_test[X_test["user_id"]==user]["item_id"].values
        reco_user = reco_all_user[i]
        for item in list_test_item_per_user:      
            if item in reco_user:
                TP += 1
            else:
                FN += 1
    precision = round((TP)/((n-1)*len(users_test_unique)),5) #menos 1 porque el n debe ser n+1
    recall = round((TP)/(TP+FN),5)
    f1 = 2*((precision * recall)/(precision + recall))
    return precision, recall, f1

# 5 recommendations

In [24]:
reco_all_user_5 = []
reco_all_user_5 = n_content_recommendation(users_test_unique, n=6, distance_matrix=cosine_sim2)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 
Progress in % 1.95 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.96 
Progress in % 1.97 
Progress in % 1.97 
Progress in % 1.98 
Progress in % 1.98 
Progress in % 1.99 
Progress in % 1.99 
Progress in % 1.99 
Progress in % 2.00 


Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 
Progress in % 3.69 
Progress in % 3.70 
Progress in % 3.70 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.72 
Progress in % 3.72 
Progress in % 3.73 
Progress in % 3.73 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.74 
Progress in % 3.75 
Progress in % 3.75 
Progress in % 3.76 
Progress in % 3.76 


Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 
Progress in % 5.50 
Progress in % 5.50 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.52 
Progress in % 5.52 
Progress in % 5.53 
Progress in % 5.53 
Progress in % 5.54 
Progress in % 5.54 
Progress in % 5.54 
Progress in % 5.55 
Progress in % 5.55 


Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.24 
Progress in % 7.24 
Progress in % 7.25 
Progress in % 7.25 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.27 
Progress in % 7.27 
Progress in % 7.28 
Progress in % 7.28 
Progress in % 7.28 
Progress in % 7.29 


...

Progress in % 91.84 
Progress in % 91.84 
Progress in % 91.85 
Progress in % 91.85 
Progress in % 91.86 
Progress in % 91.86 
Progress in % 91.86 
Progress in % 91.87 
Progress in % 91.87 
Progress in % 91.88 
Progress in % 91.88 
Progress in % 91.88 
Progress in % 91.89 
Progress in % 91.89 
Progress in % 91.90 
Progress in % 91.90 
Progress in % 91.91 
Progress in % 91.91 
Progress in % 91.91 
Progress in % 91.92 
Progress in % 91.92 
Progress in % 91.93 
Progress in % 91.93 
Progress in % 91.94 
Progress in % 91.94 
Progress in % 91.94 
Progress in % 91.95 
Progress in % 91.95 
Progress in % 91.96 
Progress in % 91.96 
Progress in % 91.96 
Progress in % 91.97 
Progress in % 91.97 
Progress in % 91.98 
Progress in % 91.98 
Progress in % 91.99 
Progress in % 91.99 
Progress in % 91.99 
Progress in % 92.00 
Progress in % 92.00 
Progress in % 92.01 
Progress in % 92.01 
Progress in % 92.02 
Progress in % 92.02 
Progress in % 92.02 
Progress in % 92.03 
Progress in % 92.03 
Progress in %

Progress in % 93.50 
Progress in % 93.51 
Progress in % 93.51 
Progress in % 93.52 
Progress in % 93.52 
Progress in % 93.52 
Progress in % 93.53 
Progress in % 93.53 
Progress in % 93.54 
Progress in % 93.54 
Progress in % 93.55 
Progress in % 93.55 
Progress in % 93.55 
Progress in % 93.56 
Progress in % 93.56 
Progress in % 93.57 
Progress in % 93.57 
Progress in % 93.58 
Progress in % 93.58 
Progress in % 93.58 
Progress in % 93.59 
Progress in % 93.59 
Progress in % 93.60 
Progress in % 93.60 
Progress in % 93.60 
Progress in % 93.61 
Progress in % 93.61 
Progress in % 93.62 
Progress in % 93.62 
Progress in % 93.63 
Progress in % 93.63 
Progress in % 93.63 
Progress in % 93.64 
Progress in % 93.64 
Progress in % 93.65 
Progress in % 93.65 
Progress in % 93.66 
Progress in % 93.66 
Progress in % 93.66 
Progress in % 93.67 
Progress in % 93.67 
Progress in % 93.68 
Progress in % 93.68 
Progress in % 93.68 
Progress in % 93.69 
Progress in % 93.69 
Progress in % 93.70 
Progress in %

Progress in % 95.23 
Progress in % 95.24 
Progress in % 95.24 
Progress in % 95.24 
Progress in % 95.25 
Progress in % 95.25 
Progress in % 95.26 
Progress in % 95.26 
Progress in % 95.27 
Progress in % 95.27 
Progress in % 95.27 
Progress in % 95.28 
Progress in % 95.28 
Progress in % 95.29 
Progress in % 95.29 
Progress in % 95.30 
Progress in % 95.30 
Progress in % 95.30 
Progress in % 95.31 
Progress in % 95.31 
Progress in % 95.32 
Progress in % 95.32 
Progress in % 95.32 
Progress in % 95.33 
Progress in % 95.33 
Progress in % 95.34 
Progress in % 95.34 
Progress in % 95.35 
Progress in % 95.35 
Progress in % 95.35 
Progress in % 95.36 
Progress in % 95.36 
Progress in % 95.37 
Progress in % 95.37 
Progress in % 95.38 
Progress in % 95.38 
Progress in % 95.38 
Progress in % 95.39 
Progress in % 95.39 
Progress in % 95.40 
Progress in % 95.40 
Progress in % 95.40 
Progress in % 95.41 
Progress in % 95.41 
Progress in % 95.42 
Progress in % 95.42 
Progress in % 95.43 
Progress in %

Progress in % 96.88 
Progress in % 96.89 
Progress in % 96.89 
Progress in % 96.90 
Progress in % 96.90 
Progress in % 96.91 
Progress in % 96.91 
Progress in % 96.91 
Progress in % 96.92 
Progress in % 96.92 
Progress in % 96.93 
Progress in % 96.93 
Progress in % 96.94 
Progress in % 96.94 
Progress in % 96.94 
Progress in % 96.95 
Progress in % 96.95 
Progress in % 96.96 
Progress in % 96.96 
Progress in % 96.96 
Progress in % 96.97 
Progress in % 96.97 
Progress in % 96.98 
Progress in % 96.98 
Progress in % 96.99 
Progress in % 96.99 
Progress in % 96.99 
Progress in % 97.00 
Progress in % 97.00 
Progress in % 97.01 
Progress in % 97.01 
Progress in % 97.02 
Progress in % 97.02 
Progress in % 97.02 
Progress in % 97.03 
Progress in % 97.03 
Progress in % 97.04 
Progress in % 97.04 
Progress in % 97.04 
Progress in % 97.05 
Progress in % 97.05 
Progress in % 97.06 
Progress in % 97.06 
Progress in % 97.07 
Progress in % 97.07 
Progress in % 97.07 
Progress in % 97.08 
Progress in %

Progress in % 98.61 
Progress in % 98.62 
Progress in % 98.62 
Progress in % 98.63 
Progress in % 98.63 
Progress in % 98.63 
Progress in % 98.64 
Progress in % 98.64 
Progress in % 98.65 
Progress in % 98.65 
Progress in % 98.66 
Progress in % 98.66 
Progress in % 98.66 
Progress in % 98.67 
Progress in % 98.67 
Progress in % 98.68 
Progress in % 98.68 
Progress in % 98.68 
Progress in % 98.69 
Progress in % 98.69 
Progress in % 98.70 
Progress in % 98.70 
Progress in % 98.71 
Progress in % 98.71 
Progress in % 98.71 
Progress in % 98.72 
Progress in % 98.72 
Progress in % 98.73 
Progress in % 98.73 
Progress in % 98.74 
Progress in % 98.74 
Progress in % 98.74 
Progress in % 98.75 
Progress in % 98.75 
Progress in % 98.76 
Progress in % 98.76 
Progress in % 98.76 
Progress in % 98.77 
Progress in % 98.77 
Progress in % 98.78 
Progress in % 98.78 
Progress in % 98.79 
Progress in % 98.79 
Progress in % 98.79 
Progress in % 98.80 
Progress in % 98.80 
Progress in % 98.81 
Progress in %

In [25]:
reco_all_user_5[1:4]

[[1547971, 1010328, 153475, 550322, 134015],
 [1106101, 1036192, 1099773, 1447941, 749329],
 [1163553, 721403, 1697200, 167112, 174086]]

In [26]:
precision_5, recall_5, f1_5 = score_metrics(n=6, reco_all_user= reco_all_user_5)

In [27]:
print(precision_5)
print(recall_5)
print(f1_5)

0.00302
0.01155
0.00478805765271105


# 10 recommendations

In [28]:
reco_all_user_10 = []
reco_all_user_10 = n_content_recommendation(users_test_unique, n=11, distance_matrix=cosine_sim2)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.73 
Progress in % 1.74 
Progress in % 1.74 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.76 
Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 


Progress in % 3.48 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.50 
Progress in % 3.50 
Progress in % 3.51 
Progress in % 3.51 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.53 
Progress in % 3.53 
Progress in % 3.54 
Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 


Progress in % 5.30 
Progress in % 5.30 
Progress in % 5.31 
Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 
Progress in % 5.50 
Progress in % 5.50 
Progress in % 5.51 


Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.24 
Progress in % 7.24 
Progress in % 7.25 
Progress in % 7.25 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.27 
Progress in % 7.27 
Progress in % 7.28 
Progress in % 7.28 
Progress in % 7.28 
Progress in % 7.29 
Progress in % 7.29 
Progress in % 7.30 


...

Progress in % 91.91 
Progress in % 91.91 
Progress in % 91.92 
Progress in % 91.92 
Progress in % 91.93 
Progress in % 91.93 
Progress in % 91.94 
Progress in % 91.94 
Progress in % 91.94 
Progress in % 91.95 
Progress in % 91.95 
Progress in % 91.96 
Progress in % 91.96 
Progress in % 91.96 
Progress in % 91.97 
Progress in % 91.97 
Progress in % 91.98 
Progress in % 91.98 
Progress in % 91.99 
Progress in % 91.99 
Progress in % 91.99 
Progress in % 92.00 
Progress in % 92.00 
Progress in % 92.01 
Progress in % 92.01 
Progress in % 92.02 
Progress in % 92.02 
Progress in % 92.02 
Progress in % 92.03 
Progress in % 92.03 
Progress in % 92.04 
Progress in % 92.04 
Progress in % 92.05 
Progress in % 92.05 
Progress in % 92.05 
Progress in % 92.06 
Progress in % 92.06 
Progress in % 92.07 
Progress in % 92.07 
Progress in % 92.07 
Progress in % 92.08 
Progress in % 92.08 
Progress in % 92.09 
Progress in % 92.09 
Progress in % 92.10 
Progress in % 92.10 
Progress in % 92.10 
Progress in %

Progress in % 93.55 
Progress in % 93.56 
Progress in % 93.56 
Progress in % 93.57 
Progress in % 93.57 
Progress in % 93.58 
Progress in % 93.58 
Progress in % 93.58 
Progress in % 93.59 
Progress in % 93.59 
Progress in % 93.60 
Progress in % 93.60 
Progress in % 93.60 
Progress in % 93.61 
Progress in % 93.61 
Progress in % 93.62 
Progress in % 93.62 
Progress in % 93.63 
Progress in % 93.63 
Progress in % 93.63 
Progress in % 93.64 
Progress in % 93.64 
Progress in % 93.65 
Progress in % 93.65 
Progress in % 93.66 
Progress in % 93.66 
Progress in % 93.66 
Progress in % 93.67 
Progress in % 93.67 
Progress in % 93.68 
Progress in % 93.68 
Progress in % 93.68 
Progress in % 93.69 
Progress in % 93.69 
Progress in % 93.70 
Progress in % 93.70 
Progress in % 93.71 
Progress in % 93.71 
Progress in % 93.71 
Progress in % 93.72 
Progress in % 93.72 
Progress in % 93.73 
Progress in % 93.73 
Progress in % 93.74 
Progress in % 93.74 
Progress in % 93.74 
Progress in % 93.75 
Progress in %

Progress in % 95.28 
Progress in % 95.29 
Progress in % 95.29 
Progress in % 95.30 
Progress in % 95.30 
Progress in % 95.30 
Progress in % 95.31 
Progress in % 95.31 
Progress in % 95.32 
Progress in % 95.32 
Progress in % 95.32 
Progress in % 95.33 
Progress in % 95.33 
Progress in % 95.34 
Progress in % 95.34 
Progress in % 95.35 
Progress in % 95.35 
Progress in % 95.35 
Progress in % 95.36 
Progress in % 95.36 
Progress in % 95.37 
Progress in % 95.37 
Progress in % 95.38 
Progress in % 95.38 
Progress in % 95.38 
Progress in % 95.39 
Progress in % 95.39 
Progress in % 95.40 
Progress in % 95.40 
Progress in % 95.40 
Progress in % 95.41 
Progress in % 95.41 
Progress in % 95.42 
Progress in % 95.42 
Progress in % 95.43 
Progress in % 95.43 
Progress in % 95.43 
Progress in % 95.44 
Progress in % 95.44 
Progress in % 95.45 
Progress in % 95.45 
Progress in % 95.46 
Progress in % 95.46 
Progress in % 95.46 
Progress in % 95.47 
Progress in % 95.47 
Progress in % 95.48 
Progress in %

Progress in % 96.93 
Progress in % 96.94 
Progress in % 96.94 
Progress in % 96.94 
Progress in % 96.95 
Progress in % 96.95 
Progress in % 96.96 
Progress in % 96.96 
Progress in % 96.96 
Progress in % 96.97 
Progress in % 96.97 
Progress in % 96.98 
Progress in % 96.98 
Progress in % 96.99 
Progress in % 96.99 
Progress in % 96.99 
Progress in % 97.00 
Progress in % 97.00 
Progress in % 97.01 
Progress in % 97.01 
Progress in % 97.02 
Progress in % 97.02 
Progress in % 97.02 
Progress in % 97.03 
Progress in % 97.03 
Progress in % 97.04 
Progress in % 97.04 
Progress in % 97.04 
Progress in % 97.05 
Progress in % 97.05 
Progress in % 97.06 
Progress in % 97.06 
Progress in % 97.07 
Progress in % 97.07 
Progress in % 97.07 
Progress in % 97.08 
Progress in % 97.08 
Progress in % 97.09 
Progress in % 97.09 
Progress in % 97.10 
Progress in % 97.10 
Progress in % 97.10 
Progress in % 97.11 
Progress in % 97.11 
Progress in % 97.12 
Progress in % 97.12 
Progress in % 97.12 
Progress in %

Progress in % 98.58 
Progress in % 98.59 
Progress in % 98.59 
Progress in % 98.60 
Progress in % 98.60 
Progress in % 98.60 
Progress in % 98.61 
Progress in % 98.61 
Progress in % 98.62 
Progress in % 98.62 
Progress in % 98.63 
Progress in % 98.63 
Progress in % 98.63 
Progress in % 98.64 
Progress in % 98.64 
Progress in % 98.65 
Progress in % 98.65 
Progress in % 98.66 
Progress in % 98.66 
Progress in % 98.66 
Progress in % 98.67 
Progress in % 98.67 
Progress in % 98.68 
Progress in % 98.68 
Progress in % 98.68 
Progress in % 98.69 
Progress in % 98.69 
Progress in % 98.70 
Progress in % 98.70 
Progress in % 98.71 
Progress in % 98.71 
Progress in % 98.71 
Progress in % 98.72 
Progress in % 98.72 
Progress in % 98.73 
Progress in % 98.73 
Progress in % 98.74 
Progress in % 98.74 
Progress in % 98.74 
Progress in % 98.75 
Progress in % 98.75 
Progress in % 98.76 
Progress in % 98.76 
Progress in % 98.76 
Progress in % 98.77 
Progress in % 98.77 
Progress in % 98.78 
Progress in %

In [29]:
reco_all_user_10[1:4]

[[1547971,
  1010328,
  153475,
  550322,
  134015,
  127865,
  638318,
  125564,
  139792,
  1859768],
 [1106101,
  1036192,
  1099773,
  1447941,
  749329,
  1872602,
  531077,
  1603811,
  1429859,
  1207456],
 [1163553,
  721403,
  1697200,
  167112,
  174086,
  1238932,
  1308013,
  161541,
  193179,
  1566348]]

In [30]:
precision_10, recall_10, f1_10 = score_metrics(n=11, reco_all_user= reco_all_user_10)

In [31]:
print(precision_10)
print(recall_10)
print(f1_10)

0.00263
0.0201
0.0046513858336999555


# 20 recommendations

In [32]:
reco_all_user_20 = []
reco_all_user_20= n_content_recommendation(users_test_unique, n=21, distance_matrix=cosine_sim2)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.74 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.76 
Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 


Progress in % 3.47 
Progress in % 3.47 
Progress in % 3.48 
Progress in % 3.48 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.49 
Progress in % 3.50 
Progress in % 3.50 
Progress in % 3.51 
Progress in % 3.51 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.53 
Progress in % 3.53 
Progress in % 3.54 
Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 


Progress in % 5.19 
Progress in % 5.20 
Progress in % 5.20 
Progress in % 5.21 
Progress in % 5.21 
Progress in % 5.21 
Progress in % 5.22 
Progress in % 5.22 
Progress in % 5.23 
Progress in % 5.23 
Progress in % 5.24 
Progress in % 5.24 
Progress in % 5.24 
Progress in % 5.25 
Progress in % 5.25 
Progress in % 5.26 
Progress in % 5.26 
Progress in % 5.27 
Progress in % 5.27 
Progress in % 5.27 
Progress in % 5.28 
Progress in % 5.28 
Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.29 
Progress in % 5.30 
Progress in % 5.30 
Progress in % 5.31 
Progress in % 5.31 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 


Progress in % 6.93 
Progress in % 6.93 
Progress in % 6.93 
Progress in % 6.94 
Progress in % 6.94 
Progress in % 6.95 
Progress in % 6.95 
Progress in % 6.96 
Progress in % 6.96 
Progress in % 6.96 
Progress in % 6.97 
Progress in % 6.97 
Progress in % 6.98 
Progress in % 6.98 
Progress in % 6.99 
Progress in % 6.99 
Progress in % 6.99 
Progress in % 7.00 
Progress in % 7.00 
Progress in % 7.01 
Progress in % 7.01 
Progress in % 7.01 
Progress in % 7.02 
Progress in % 7.02 
Progress in % 7.03 
Progress in % 7.03 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.04 
Progress in % 7.05 
Progress in % 7.05 
Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 


...

Progress in % 91.58 
Progress in % 91.58 
Progress in % 91.59 
Progress in % 91.59 
Progress in % 91.59 
Progress in % 91.60 
Progress in % 91.60 
Progress in % 91.61 
Progress in % 91.61 
Progress in % 91.62 
Progress in % 91.62 
Progress in % 91.62 
Progress in % 91.63 
Progress in % 91.63 
Progress in % 91.64 
Progress in % 91.64 
Progress in % 91.64 
Progress in % 91.65 
Progress in % 91.65 
Progress in % 91.66 
Progress in % 91.66 
Progress in % 91.67 
Progress in % 91.67 
Progress in % 91.67 
Progress in % 91.68 
Progress in % 91.68 
Progress in % 91.69 
Progress in % 91.69 
Progress in % 91.70 
Progress in % 91.70 
Progress in % 91.70 
Progress in % 91.71 
Progress in % 91.71 
Progress in % 91.72 
Progress in % 91.72 
Progress in % 91.72 
Progress in % 91.73 
Progress in % 91.73 
Progress in % 91.74 
Progress in % 91.74 
Progress in % 91.75 
Progress in % 91.75 
Progress in % 91.75 
Progress in % 91.76 
Progress in % 91.76 
Progress in % 91.77 
Progress in % 91.77 
Progress in %

Progress in % 93.23 
Progress in % 93.24 
Progress in % 93.24 
Progress in % 93.25 
Progress in % 93.25 
Progress in % 93.25 
Progress in % 93.26 
Progress in % 93.26 
Progress in % 93.27 
Progress in % 93.27 
Progress in % 93.28 
Progress in % 93.28 
Progress in % 93.28 
Progress in % 93.29 
Progress in % 93.29 
Progress in % 93.30 
Progress in % 93.30 
Progress in % 93.31 
Progress in % 93.31 
Progress in % 93.31 
Progress in % 93.32 
Progress in % 93.32 
Progress in % 93.33 
Progress in % 93.33 
Progress in % 93.34 
Progress in % 93.34 
Progress in % 93.34 
Progress in % 93.35 
Progress in % 93.35 
Progress in % 93.36 
Progress in % 93.36 
Progress in % 93.36 
Progress in % 93.37 
Progress in % 93.37 
Progress in % 93.38 
Progress in % 93.38 
Progress in % 93.39 
Progress in % 93.39 
Progress in % 93.39 
Progress in % 93.40 
Progress in % 93.40 
Progress in % 93.41 
Progress in % 93.41 
Progress in % 93.42 
Progress in % 93.42 
Progress in % 93.42 
Progress in % 93.43 
Progress in %

Progress in % 94.93 
Progress in % 94.93 
Progress in % 94.94 
Progress in % 94.94 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.96 
Progress in % 94.96 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.98 
Progress in % 94.98 
Progress in % 94.99 
Progress in % 94.99 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.01 
Progress in % 95.01 
Progress in % 95.02 
Progress in % 95.02 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.04 
Progress in % 95.04 
Progress in % 95.05 
Progress in % 95.05 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.07 
Progress in % 95.07 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.09 
Progress in % 95.09 
Progress in % 95.10 
Progress in % 95.10 
Progress in % 95.11 
Progress in % 95.11 
Progress in % 95.11 
Progress in % 95.12 
Progress in % 95.12 
Progress in %

Progress in % 96.64 
Progress in % 96.65 
Progress in % 96.65 
Progress in % 96.66 
Progress in % 96.66 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.68 
Progress in % 96.68 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.70 
Progress in % 96.70 
Progress in % 96.71 
Progress in % 96.71 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.73 
Progress in % 96.73 
Progress in % 96.74 
Progress in % 96.74 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.76 
Progress in % 96.76 
Progress in % 96.77 
Progress in % 96.77 
Progress in % 96.78 
Progress in % 96.78 
Progress in % 96.78 
Progress in % 96.79 
Progress in % 96.79 
Progress in % 96.80 
Progress in % 96.80 
Progress in % 96.80 
Progress in % 96.81 
Progress in % 96.81 
Progress in % 96.82 
Progress in % 96.82 
Progress in % 96.83 
Progress in % 96.83 
Progress in % 96.83 
Progress in % 96.84 
Progress in %

Progress in % 98.35 
Progress in % 98.35 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.37 
Progress in % 98.37 
Progress in % 98.38 
Progress in % 98.38 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.40 
Progress in % 98.40 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.42 
Progress in % 98.42 
Progress in % 98.43 
Progress in % 98.43 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.45 
Progress in % 98.45 
Progress in % 98.46 
Progress in % 98.46 
Progress in % 98.47 
Progress in % 98.47 
Progress in % 98.47 
Progress in % 98.48 
Progress in % 98.48 
Progress in % 98.49 
Progress in % 98.49 
Progress in % 98.50 
Progress in % 98.50 
Progress in % 98.50 
Progress in % 98.51 
Progress in % 98.51 
Progress in % 98.52 
Progress in % 98.52 
Progress in % 98.52 
Progress in % 98.53 
Progress in % 98.53 
Progress in % 98.54 
Progress in % 98.54 
Progress in %

In [33]:
precision_20, recall_20, f1_20 = score_metrics(n=21, reco_all_user= reco_all_user_20)

In [34]:
print(precision_20)
print(recall_20)
print(f1_20)

0.0021
0.03219
0.0039427821522309715


# 50 recommendations

In [35]:
reco_all_user_50 = []
reco_all_user_50 = n_content_recommendation(users_test_unique, n=51, distance_matrix=cosine_sim2)

Progress in % 0.00 
Progress in % 0.01 
Progress in % 0.01 
Progress in % 0.02 
Progress in % 0.02 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.03 
Progress in % 0.04 
Progress in % 0.04 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.05 
Progress in % 0.06 
Progress in % 0.06 
Progress in % 0.07 
Progress in % 0.07 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.08 
Progress in % 0.09 
Progress in % 0.09 
Progress in % 0.10 
Progress in % 0.10 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.11 
Progress in % 0.12 
Progress in % 0.12 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.13 
Progress in % 0.14 
Progress in % 0.14 
Progress in % 0.15 
Progress in % 0.15 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.16 
Progress in % 0.17 
Progress in % 0.17 
Progress in % 0.18 
Progress in % 0.18 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.19 
Progress in % 0.20 
Progress in % 0.20 
Progress in % 0.21 
Progress in % 0.21 


Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.75 
Progress in % 1.76 
Progress in % 1.76 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.77 
Progress in % 1.78 
Progress in % 1.78 
Progress in % 1.79 
Progress in % 1.79 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.80 
Progress in % 1.81 
Progress in % 1.81 
Progress in % 1.82 
Progress in % 1.82 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.83 
Progress in % 1.84 
Progress in % 1.84 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.85 
Progress in % 1.86 
Progress in % 1.86 
Progress in % 1.87 
Progress in % 1.87 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.88 
Progress in % 1.89 
Progress in % 1.89 
Progress in % 1.90 
Progress in % 1.90 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.91 
Progress in % 1.92 
Progress in % 1.92 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.93 
Progress in % 1.94 
Progress in % 1.94 
Progress in % 1.95 
Progress in % 1.95 


Progress in % 3.51 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.52 
Progress in % 3.53 
Progress in % 3.53 
Progress in % 3.54 
Progress in % 3.54 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.55 
Progress in % 3.56 
Progress in % 3.56 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.57 
Progress in % 3.58 
Progress in % 3.58 
Progress in % 3.59 
Progress in % 3.59 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.60 
Progress in % 3.61 
Progress in % 3.61 
Progress in % 3.62 
Progress in % 3.62 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.63 
Progress in % 3.64 
Progress in % 3.64 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.65 
Progress in % 3.66 
Progress in % 3.66 
Progress in % 3.67 
Progress in % 3.67 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.68 
Progress in % 3.69 
Progress in % 3.69 
Progress in % 3.70 
Progress in % 3.70 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.71 
Progress in % 3.72 


Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.32 
Progress in % 5.33 
Progress in % 5.33 
Progress in % 5.34 
Progress in % 5.34 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.35 
Progress in % 5.36 
Progress in % 5.36 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.37 
Progress in % 5.38 
Progress in % 5.38 
Progress in % 5.39 
Progress in % 5.39 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.40 
Progress in % 5.41 
Progress in % 5.41 
Progress in % 5.42 
Progress in % 5.42 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.43 
Progress in % 5.44 
Progress in % 5.44 
Progress in % 5.45 
Progress in % 5.45 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.46 
Progress in % 5.47 
Progress in % 5.47 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.48 
Progress in % 5.49 
Progress in % 5.49 
Progress in % 5.50 
Progress in % 5.50 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.51 
Progress in % 5.52 
Progress in % 5.52 


Progress in % 7.06 
Progress in % 7.06 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.07 
Progress in % 7.08 
Progress in % 7.08 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.09 
Progress in % 7.10 
Progress in % 7.10 
Progress in % 7.11 
Progress in % 7.11 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.12 
Progress in % 7.13 
Progress in % 7.13 
Progress in % 7.14 
Progress in % 7.14 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.15 
Progress in % 7.16 
Progress in % 7.16 
Progress in % 7.17 
Progress in % 7.17 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.18 
Progress in % 7.19 
Progress in % 7.19 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.20 
Progress in % 7.21 
Progress in % 7.21 
Progress in % 7.22 
Progress in % 7.22 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.23 
Progress in % 7.24 
Progress in % 7.24 
Progress in % 7.25 
Progress in % 7.25 
Progress in % 7.26 
Progress in % 7.26 
Progress in % 7.26 


...

Progress in % 93.25 
Progress in % 93.26 
Progress in % 93.26 
Progress in % 93.27 
Progress in % 93.27 
Progress in % 93.28 
Progress in % 93.28 
Progress in % 93.28 
Progress in % 93.29 
Progress in % 93.29 
Progress in % 93.30 
Progress in % 93.30 
Progress in % 93.31 
Progress in % 93.31 
Progress in % 93.31 
Progress in % 93.32 
Progress in % 93.32 
Progress in % 93.33 
Progress in % 93.33 
Progress in % 93.34 
Progress in % 93.34 
Progress in % 93.34 
Progress in % 93.35 
Progress in % 93.35 
Progress in % 93.36 
Progress in % 93.36 
Progress in % 93.36 
Progress in % 93.37 
Progress in % 93.37 
Progress in % 93.38 
Progress in % 93.38 
Progress in % 93.39 
Progress in % 93.39 
Progress in % 93.39 
Progress in % 93.40 
Progress in % 93.40 
Progress in % 93.41 
Progress in % 93.41 
Progress in % 93.42 
Progress in % 93.42 
Progress in % 93.42 
Progress in % 93.43 
Progress in % 93.43 
Progress in % 93.44 
Progress in % 93.44 
Progress in % 93.44 
Progress in % 93.45 
Progress in %

Progress in % 94.90 
Progress in % 94.90 
Progress in % 94.91 
Progress in % 94.91 
Progress in % 94.92 
Progress in % 94.92 
Progress in % 94.92 
Progress in % 94.93 
Progress in % 94.93 
Progress in % 94.94 
Progress in % 94.94 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.95 
Progress in % 94.96 
Progress in % 94.96 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.97 
Progress in % 94.98 
Progress in % 94.98 
Progress in % 94.99 
Progress in % 94.99 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.00 
Progress in % 95.01 
Progress in % 95.01 
Progress in % 95.02 
Progress in % 95.02 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.03 
Progress in % 95.04 
Progress in % 95.04 
Progress in % 95.05 
Progress in % 95.05 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.06 
Progress in % 95.07 
Progress in % 95.07 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.08 
Progress in % 95.09 
Progress in % 95.09 
Progress in %

Progress in % 96.56 
Progress in % 96.57 
Progress in % 96.57 
Progress in % 96.58 
Progress in % 96.58 
Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.59 
Progress in % 96.60 
Progress in % 96.60 
Progress in % 96.61 
Progress in % 96.61 
Progress in % 96.61 
Progress in % 96.62 
Progress in % 96.62 
Progress in % 96.63 
Progress in % 96.63 
Progress in % 96.64 
Progress in % 96.64 
Progress in % 96.64 
Progress in % 96.65 
Progress in % 96.65 
Progress in % 96.66 
Progress in % 96.66 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.67 
Progress in % 96.68 
Progress in % 96.68 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.69 
Progress in % 96.70 
Progress in % 96.70 
Progress in % 96.71 
Progress in % 96.71 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.72 
Progress in % 96.73 
Progress in % 96.73 
Progress in % 96.74 
Progress in % 96.74 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.75 
Progress in % 96.76 
Progress in %

Progress in % 98.28 
Progress in % 98.28 
Progress in % 98.28 
Progress in % 98.29 
Progress in % 98.29 
Progress in % 98.30 
Progress in % 98.30 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.31 
Progress in % 98.32 
Progress in % 98.32 
Progress in % 98.33 
Progress in % 98.33 
Progress in % 98.33 
Progress in % 98.34 
Progress in % 98.34 
Progress in % 98.35 
Progress in % 98.35 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.36 
Progress in % 98.37 
Progress in % 98.37 
Progress in % 98.38 
Progress in % 98.38 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.39 
Progress in % 98.40 
Progress in % 98.40 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.41 
Progress in % 98.42 
Progress in % 98.42 
Progress in % 98.43 
Progress in % 98.43 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.44 
Progress in % 98.45 
Progress in % 98.45 
Progress in % 98.46 
Progress in % 98.46 
Progress in % 98.47 
Progress in % 98.47 
Progress in %

Progress in % 99.94 
Progress in % 99.95 
Progress in % 99.95 
Progress in % 99.95 
Progress in % 99.96 
Progress in % 99.96 
Progress in % 99.97 
Progress in % 99.97 
Progress in % 99.97 
Progress in % 99.98 
Progress in % 99.98 
Progress in % 99.99 
Progress in % 99.99 
Progress in % 100.00 
Progress in % 100.00 


In [36]:
precision_50, recall_50, f1_50 = score_metrics(n=51, reco_all_user= reco_all_user_50)

In [37]:
print(precision_50)
print(recall_50)
print(f1_50)

0.00163
0.06248
0.0031771143347371703


# Resultados

In [38]:
resultados = {'n_recomendaciones': [5, 10, 20, 50],
              'hybrid_recall': [recall_5,
                            recall_10,
                            recall_20,
                            recall_50],
              'hybrid_precision': [precision_5,
                               precision_10,
                               precision_20,
                               precision_50],
              'hybrid_F1': [f1_5,
                            f1_10,
                            f1_20,
                            f1_50]
                    }

In [39]:
resultados_df = pd.DataFrame.from_dict(resultados)
resultados_df

,n_recomendaciones,hybrid_recall,hybrid_precision,hybrid_F1
0,5,0.01155,0.00302,0.004788
1,10,0.02010,0.00263,0.004651
2,20,0.03219,0.00210,0.003943
3,50,0.06248,0.00163,0.003177


In [40]:
resultados_df.to_csv("resultados_content_soup.csv",index=False)

# Ejemplo con un caso

In [41]:
df_test

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,large,495160,34a,216048,108lbs,8.0,everyday,I very comfortable casual dress; I wore it sho...,petite,Comfortable casual dress,dress,"5' 2""",1,36.0,"May 19, 2016"
1,fit,510655,34c,154002,125lbs,10.0,formal affair,I wore this to a charity ball I chaired so I w...,hourglass,Super glamorous,gown,"5' 5""",8,56.0,"April 16, 2015"
2,fit,974819,38b,1111981,150lbs,10.0,wedding,Dress runs a tad small in the waist and torso....,athletic,Beautiful Dress,dress,"5' 7""",20,66.0,"October 9, 2015"
3,small,704655,34d,734229,155lbs,8.0,other,"Loved this dress, felt it ran a little small. ...",hourglass,Perfect for Bridal Shower,dress,"5' 5""",20,32.0,"September 12, 2017"
4,fit,312825,32a,512791,121lbs,10.0,work,fyi,petite,loved it! so comfortable and suave,dress,"5' 2""",8,48.0,"October 4, 2017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33919,large,731568,36d,1353012,175lbs,10.0,formal affair,I loved this dress - very unique and comfortab...,hourglass,Unique and comfortable,gown,"5' 8""",24,36.0,"January 11, 2017"
33920,small,377141,36d,1678888,150lbs,10.0,party,I love this dress will definitely rent again. ...,athletic,Show off your curves,dress,"5' 6""",20,28.0,"January 19, 2017"
33921,fit,220884,NaN,1172052,NaN,8.0,formal affair,I'm 5'9 and a true size 22. This dress fit gre...,full bust,Perfect for a black and white themed gala,sheath,"5' 9""",57,27.0,"October 19, 2017"
33922,small,431013,32a,1764436,120lbs,8.0,work,"It's very flattering on my small on top, bigge...",hourglass,Wore this for a board meeting and everything w...,sheath,"5' 6""",4,34.0,"May 14, 2017"


In [42]:
#Usuario  con item
caso = df_test[(df_test["user_id"] ==377141) & (df_test["item_id"] == 2334570)]
caso.loc[:,["user_id", "item_id","rating", "review_summary"]]

,user_id,item_id,rating,review_summary
9900,377141,2334570,10.0,Super cute and comfy


In [43]:
#Me guardo las palabras asociadas a ese item en train
reco_1 = new_df_soup[new_df_soup['item_id']==2334570].soup.values
reco_1

array([' party jumpsuit athletic party jumpsuit athletic work jumpsuit hourglass party jumpsuit petite party jumpsuit hourglass date jumpsuit petite party jumpsuit hourglass party jumpsuit hourglass other jumpsuit athletic party jumpsuit hourglass wedding jumpsuit petite party jumpsuit petite party jumpsuit petite party jumpsuit pear wedding jumpsuit athletic date jumpsuit straight & narrow date jumpsuit athletic other jumpsuit athletic work jumpsuit pear party jumpsuit hourglass other jumpsuit athletic'],
      dtype=object)

In [44]:
results = Counter(reco_1[0].split())
results

Counter({'party': 11,
         'jumpsuit': 21,
         'athletic': 7,
         'work': 2,
         'hourglass': 6,
         'petite': 5,
         'date': 3,
         'other': 3,
         'wedding': 2,
         'pear': 2,
         'straight': 1,
         '&': 1,
         'narrow': 1})

In [45]:
#Hago recomendacion sobre ese item
content_recommender(2334570)

NameError: name 'content_recommender' is not defined

In [ ]:
#Observo la composicion del item recomendado
reco_2 = new_df_soup[new_df_soup['item_id']==2155094].soup.values

In [ ]:
results = Counter(reco_2[0].split())
results